In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 약물
추출된 전체 약물 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_drugs():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['name']

In [7]:
drugs['refined'] = ''
drugs.head()

,name,refined
0,edoxaban,
1,levofloxacin-non-susceptible,
2,rivastigmine,
3,procarbazine,
4,bambuterol,


#### 전처리 전 총 약물의 수는 2323개 입니다

In [8]:
drugs.shape

(2323, 2)

### 2. 전처리

#### A. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

In [9]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-prev', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-tolerance', '-facilitated', '-replacement', '-to-renin', '-gits', \
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '-insensitive',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen'
                     ]

In [10]:
# .....'-the', 'on-', 
_replace_with_space = ['-receptor'
                      'post-','-dose-dense', '–placebo', \
                      '-inhibiting', '-containing', '-related', '-like', '-boosted', '-binding', '8 × ', '-only', \
                      '-pulse', '-positive', '-confirmed', '-controlled', '-releasing', '-lowering', 'pre-', '≥110', \
                      '-independent', '≥5', 'secreting', 'mothers', 'bothersome', "mothers'", 'senegal']

# '-dependent', , 

In [11]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')

drugs = drugs.reset_index(drop=True)

edoxaban
levofloxacin-non-susceptible
levofloxacin 
rivastigmine
procarbazine
bambuterol
metyrapone
cyproheptadine
cyclobenzaprine
pseudo-cholinesterase
famotidine
isepamicin
inductionciclosporin
insulin lispro
lamotrigine-high
lamotrigine 
dulaglutide
acidlevamisole
deutetrabenazine
etanercept
bevacizumab.
triamterene
fluorescein
ceftazidime
isoniazidthiacetazone
low-density-lipoprotein c
tamoxifencmf33
aliskiren
azacitidine
flupirtine
oxygenator
hyaluronic acid
belimumab
pomalidomide
exenatide
vinblastine-involved-field
vinblastine 
sulfadoxine-pyrimethamine
heparinglycoprotein
rifampicinpyrazinamide
emtricitabine
1-sotalol
acetylcarnitine
busulfan-based
busulfan 
collagenase
sotalol
fructose
non-secretin
albumin
ezetimibe<0.001
endothelium
others.
carboplatin
gentamycinemetronidazole
hydrochlorothiazide
rosiglitazone
albumin:creatinine
high-density-lipoprotein cholesterol
albendazole.
von willebrand factor
nadolol
enalapril-studies
enalapril 
treosulfan-tt-fludarabine
canagliflozin


protein c2
chlorhexidine
nateglinide
chlorothiazide
caspofungin
rimonabant-the
vinblastine
pantoprazole
heparin-
plerixafor
cocaine-negative
oxaliplatin-based
oxaliplatin 
pemetrexed-based
pemetrexed 
urapidil
lacidipine
nimodipine
alectinib
dronedarone
propanol
methionine-threonine
bivalirudinglycoprotein
bivalirudin
gemcitabine-dacarbazine
fluvastatin
carvedilol-metoprolol
s-warfarin
metronidazole
tegafurother
amrubicin
caffeine-ip-ischemic
caffeine-ip 
sevelamer
agomelatine
opium
bendamustine.
rituximab-doxorubicin
amikacin
calcium gluconate
atenololchlorthalidone
moxonidine
amifostine
clomipramine
medrogestone
methotrexate-based
methotrexate 
rosuvastatin-treated
rosuvastatin 
vorinostat
bp-sirolimus-eluting
bp-sirolimus 
melphalan.
granisetron
venlafaxine
daunorubicin
cefamandolemetronidazole
doxorubicin-ifosfamide
solifenacin
127artemether
glucose]
glucose 
omeprazole
ethionamide
perphenazine
tetracycline-resistant
tetracycline 
nintedanib
glucose-6-phosphate
irinotecanfluorourac

adalimumab 
cisapride
codeine
aclarubicin
30-albumin
carbachol
diethylcarbamazine
l-asparaginase-containing
l-asparaginase 
n-phenylacetyl-l-glutamine
placebo+prednisone
sodium fluoride
crizotinib-naive
crizotinib 
treprostinil
no-irinotecan
cabozantinib
post-enzalutamide
enfuvirtide-naive
enfuvirtide 
leflunomide
benazepril
amino acids
hydroxychloroquine
tetracosactide
warfarin-naive
warfarin 
fenoterol
cariprazine
minocycline
clonazepam
acitretin
rofecoxib
ataluren
inosine
goserelin
dasatinib
galactose-1-phosphate
abciximab-sirolimus-eluting
abciximab-sirolimus 
thiomersal-exposed
ranolazine-lidocaine
glucosamine
carvedilolmetoprolol
fumaric acid
trimethoprim
cefaclor
lenvatinib
eculizumab
rituximab-ibritumomab
artemether.
oxazepam
zolpidem
ifosfamide-carboplatin-etoposide
gamma-tocopherol
rifapentine
cobimetinib
apremilast
thalidomide
valsartan.
fenbufen
tc-sulfur
etanercept.
valdecoxib
enoximone
blood-glucose-lowering
lamotrigine-extended
lamotrigine 
5-fluorouracil
iodine-123-beta

felbamate
tofacitinib
nitrofural
roxithromycin
bendamustine-containing
bendamustine 
phenformin
-biotin
delapril
betaxolol
trimethoprim-sulfamethoxazolemeropenem
primidone.
bezafibrate
ursodeoxycholic acid
stavudinelamivudine
beraprost
fotemustine
carvedilol.
ciprofloxacinceftazidime
nefopam
everolimus-2
minoxidil
demeclocycline
carvedilol
chloroquine-sensitive
chloroquine 
infant-nevirapine
nitrogen-containing
nitrogen 
thrombin&#8211
oxaliplatinfluorouracil
sirolimus-coated
sirolimus 
liraglutide–exenatide
serotonin-norepinephrine
oxaliplatin:
prasugrel
ciclosporin
pre-propafenone
trospium
enzalutamide
carboplatin.
cefepime
cefotaxime
week+oxaliplatin
ery-methotrexate
α1-antitrypsin
mepacrine
microalbuminuria.
raltitrexed
amyl nitrites
omeprazole:
levocetirizine
interferon-gamma.
trimethoprim-sulfamethoxazole
vincristine-associated
vincristine 
nitrofurantoin
amoxicillin
cetuximab
topiramate
trifluoperazine
prazosin
5-flucytosine
prednisone
ofloxacin
pegloticase-related:
pegloticase 

fasudil
nafcillin
eplerenone
mesoridazine
donepezil
azelastine
pefloxacin
artesunate
capecitabine-based
capecitabine 
carisoprodol
apixaban
fluorouracil-levamisole
atosiban.
tirofiban.
fluphenazine
lamotrigine-low
diethylenetriaminepentaacetic acid
epirubicin-p-cyclophosphamide
non-naproxen
ixekizumab
electrolytes
microalbuminuric
rituximab-sensitive
rituximab 
tryptophan
sirolimus-eluting
sirolimus 
daclizumab
hydroxy-norclobazam
bendamustinem
insulin degludecinsulin
nortriptyline.
capecitabine-cisplatin
cefalexin
prasugrel5mgvs.prasugrel10
artesunate.
benzathine benzylpenicillin
imatinib.
cyclophosphamide
anastrozole
nitrous oxide
extended-release-verapamil.
d4t>others
hydroquinidine
tegafur
amiloride
insulin glargine
atazanavir-ritonavir
salbutamol
abciximab.
brentuximab vedotin
lamivudine.
rituximab-chop
dobutamine-
telmisartan+ramipril
oxycodone.
glucose-insulin
goserelinf
exercise-dipyridamole
trastuzumab emtansine
tetracycline
salmeterol.
ertapenem
nifurtimox-eflornithine
emtric

#### 전처리와 약물이 아닌 레코드를 제거하면 2310개가 남습니다

In [12]:
drugs.shape

(2310, 2)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [13]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting', 'non-']

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [14]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        drugs.drop(ind, inplace=True)
        print('===============================')

drugs = drugs.reset_index(drop=True)

levofloxacin-non-susceptible
pseudo-cholinesterase
non-secretin
heparin-placebo
donepezilplacebo
salmeterolplacebo
non-bevacizumab
cocaine-negative
non-cetuximab
iodine-deficient
non-amiodarone
(non-eosinophilic
nicotine-free
non-interferon-beta
salmeterol+placebo
buserelin+placebo
placebo+prednisone
tuberculin-negative
non-indinavir-containing
non-ciclosporin
placebodocetaxel
methylprednisolone-placebo
non-probucol
bevacizumab-free
non-paclitaxel-eluting
placebo-clopidogrel
non-eosinophilic
cholinesterase-inhibiting
adalimumab-withdrawal
placebodonepezil
non-protein c
aldosterone-blocking
placebomethotrexate
non-naproxen


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 2276 남습니다

In [15]:
drugs.shape

(2276, 2)

### 앞 뒤의 -, . 제거

In [16]:
remove_list = ['.', '-', ':', '+', '–', ';', '%']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
drugs.name = drugs.name.map(lambda x: preprocess(x))

### 추가 전처리 필요없는 약물들은  refined로 입력

In [17]:
spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
sep_pattern = re.compile("|".join(spe_chars.keys()))
int_pattern = re.compile('[0-9]')

for ind, row in drugs.iterrows():
    refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
    int_refined = int_pattern.search(row['name'])
    
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() == refined.strip() and int_refined is None:
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    print('===============================')

edoxaban
edoxaban
rivastigmine
rivastigmine
procarbazine
procarbazine
bambuterol
bambuterol
metyrapone
metyrapone
cyproheptadine
cyproheptadine
cyclobenzaprine
cyclobenzaprine
famotidine
famotidine
isepamicin
isepamicin
inductionciclosporin
inductionciclosporin
insulin lispro
insulin lispro
dulaglutide
dulaglutide
acidlevamisole
acidlevamisole
deutetrabenazine
deutetrabenazine
etanercept
etanercept
bevacizumab
bevacizumab
triamterene
triamterene
fluorescein
fluorescein
ceftazidime
ceftazidime
isoniazidthiacetazone
isoniazidthiacetazone
aliskiren
aliskiren
azacitidine
azacitidine
flupirtine
flupirtine
oxygenator
oxygenator
hyaluronic acid
hyaluronic acid
belimumab
belimumab
pomalidomide
pomalidomide
exenatide
exenatide
heparinglycoprotein
heparinglycoprotein
rifampicinpyrazinamide
rifampicinpyrazinamide
emtricitabine
emtricitabine
acetylcarnitine
acetylcarnitine
collagenase
collagenase
sotalol
sotalol
fructose
fructose
albumin
albumin
endothelium
endothelium
others
others
carboplatin
ca

halothane
halothane
pentoxifylline
pentoxifylline
melatonin
melatonin
interferon
interferon
mefloquine
mefloquine
saquinavir
saquinavir
trimetazidine
trimetazidine
levodopa
levodopa
trastuzumab
trastuzumab
testosterone
testosterone
bedaquiline
bedaquiline
benzatropine
benzatropine
monothioglycerol
monothioglycerol
auranofin
auranofin
etoricoxib
etoricoxib
desethylamiodarone
desethylamiodarone
abacavirlamivudine
abacavirlamivudine
foods
foods
dosulepin
dosulepin
octreotide
octreotide
erlotinib
erlotinib
azathioprine
azathioprine
ethanol
ethanol
ciclosporinazsteroids
ciclosporinazsteroids
melphalanprednisolone
melphalanprednisolone
selegiline
selegiline
rilpivirine
rilpivirine
ramipriltelmisartan
ramipriltelmisartan
sulfinpyrazone
sulfinpyrazone
asparaginase
asparaginase
methoxamine
methoxamine
hydroxycarbamide
hydroxycarbamide
celiprolol
celiprolol
apomorphine
apomorphine
prazepam
prazepam
miglitol
miglitol
lepirudin
lepirudin
mepolizumab
mepolizumab
masitinib
masitinib
glatiramer aceta

strontium ranelate
quetiapine
quetiapine
ketamine
ketamine
roflumilast
roflumilast
meclozine
meclozine
sevoflurane
sevoflurane
aldosterone
aldosterone
flecainide
flecainide
medroxyprogesterone
medroxyprogesterone
rasagiline
rasagiline
sulfasalazine
sulfasalazine
albiglutide
albiglutide
sodium phosphate
sodium phosphate
pitolisant
pitolisant
chlorpromazine
chlorpromazine
amodiaquine
amodiaquine
nystatin
nystatin
maprotiline
maprotiline
capiricetuximab
capiricetuximab
vedolizumab
vedolizumab
desonide
desonide
guanethidine
guanethidine
erythropoietins
erythropoietins
alemtuzumab
alemtuzumab
furosemide
furosemide
malelithium
malelithium
byeptifibatide
byeptifibatide
pasireotide
pasireotide
eosin
eosin
ustekinumab
ustekinumab
ajmaline
ajmaline
diamorphine
diamorphine
tenecteplase
tenecteplase
ribavirin
ribavirin
nitazoxanide
nitazoxanide
potassium chloride
potassium chloride
albuminuria
albuminuria
citalopram
citalopram
mifepristone
mifepristone
collagen
collagen
lansoprazole
lansoprazole
m

etanercept
etanercept
valdecoxib
valdecoxib
enoximone
enoximone
isoflurane
isoflurane
eslicarbazepine
eslicarbazepine
allopurinol
allopurinol
donepezildonepezil
donepezildonepezil
mannitol
mannitol
citicoline
citicoline
methadone]
methadone]
atazanavir
atazanavir
artemisinin
artemisinin
mgpotassiumdoxazosin
mgpotassiumdoxazosin
acetylcholinesterase
acetylcholinesterase
iopamidol]
iopamidol]
isoniazidpyrazinamide
isoniazidpyrazinamide
albumin
albumin
budipine
budipine
etomidate
etomidate
erythrocytes
erythrocytes
ascorbic acid
ascorbic acid
mgbendroflumethiazide
mgbendroflumethiazide
metamfetamine
metamfetamine
cannabinoids
cannabinoids
zolmitriptan
zolmitriptan
tiotropium
tiotropium
phenprocoumon
phenprocoumon
enalapril
enalapril
fluorouracil
fluorouracil
dexrazoxane
dexrazoxane
celecoxib
celecoxib
eliglustat
eliglustat
molindone
molindone
quinaprilat
quinaprilat
decitabine
decitabine
oxycodone
oxycodone
torcetrapibatorvastatin
torcetrapibatorvastatin
sertindole
sertindole
tamsulosin
t

microalbuminuria
microalbuminuria
sunitinib
sunitinib
ezetimibe
ezetimibe
defibrotide
defibrotide
bmiltefosine
bmiltefosine
enflurane
enflurane
gatifloxacin
gatifloxacin
etoposidecyclophosphamidedoxorubicin
etoposidecyclophosphamidedoxorubicin
flucloxacillin
flucloxacillin
pramlintide
pramlintide
remifentanil
remifentanil
chloroquine
chloroquine
verapamil
verapamil
mitoxantrone
mitoxantrone
log[estradiol]
log[estradiol]
mirtazapine
mirtazapine
daratumumab
daratumumab
pivampicillin
pivampicillin
retigabine
retigabine
entecavir
entecavir
nifedipine
nifedipine
pyrimethamine
pyrimethamine
clodronic acid
clodronic acid
dextromethorphan
dextromethorphan
animals
animals
pyridoxine
pyridoxine
glimepiride
glimepiride
urokinase
urokinase
cefuroxime
cefuroxime
riluzole
riluzole
sotalol
sotalol
rivastigmine
rivastigmine
ifosfamide
ifosfamide
nicorandil
nicorandil
omalizumab
omalizumab
nicorandil
nicorandil
sulfur
sulfur
eosinophils
eosinophils
telmisartanramipril
telmisartanramipril
cyanocobalamin

chlortalidone
chlortalidone
atomoxetine
atomoxetine
rotigotine
rotigotine
rimantadine
rimantadine
colesevelam
colesevelam
iopentol
iopentol
nilvadipine
nilvadipine
eosinophil
eosinophil
metolazone
metolazone
desflurane
desflurane
ibandronic acid
ibandronic acid
mitomycin
mitomycin
iodine
iodine
protein c
protein c
furazolidone
furazolidone
dexfenfluramine
dexfenfluramine
isoprinosine
isoprinosine
amitriptyline
amitriptyline
dopamine
dopamine
oxybutynin
oxybutynin
acebutolol
acebutolol
ixabepilone
ixabepilone
cefazolinmetronidazole
cefazolinmetronidazole
heparins
heparins
verapamil
verapamil
acetazolamide
acetazolamide
temozolomide
temozolomide
calcium chloride
calcium chloride
anagrelide
anagrelide
interferon
interferon
bendamustine
bendamustine
papaverine
papaverine
nonprotamine
nonprotamine
alum
alum
sparfloxacin
sparfloxacin
nitroprusside
nitroprusside
paliperidone
paliperidone
duloxetine
duloxetine
nitric oxide
nitric oxide
glutamine
glutamine
mibefradil
mibefradil
cyclooxygenase
c

azelastine
pefloxacin
pefloxacin
artesunate
artesunate
carisoprodol
carisoprodol
apixaban
apixaban
atosiban
atosiban
tirofiban
tirofiban
fluphenazine
fluphenazine
diethylenetriaminepentaacetic acid
diethylenetriaminepentaacetic acid
ixekizumab
ixekizumab
electrolytes
electrolytes
microalbuminuric
microalbuminuric
tryptophan
tryptophan
daclizumab
daclizumab
bendamustinem
bendamustinem
insulin degludecinsulin
insulin degludecinsulin
nortriptyline
nortriptyline
cefalexin
cefalexin
artesunate
artesunate
benzathine benzylpenicillin
benzathine benzylpenicillin
imatinib
imatinib
cyclophosphamide
cyclophosphamide
anastrozole
anastrozole
nitrous oxide
nitrous oxide
hydroquinidine
hydroquinidine
tegafur
tegafur
amiloride
amiloride
insulin glargine
insulin glargine
salbutamol
salbutamol
abciximab
abciximab
brentuximab vedotin
brentuximab vedotin
lamivudine
lamivudine
dobutamine
dobutamine
oxycodone
oxycodone
goserelinf
goserelinf
trastuzumab emtansine
trastuzumab emtansine
tetracycline
tetracycli

## 선생님께서 제안해주신 추가 정제 단어 확인 작업

In [19]:
drugs.loc[drugs.name.str.find('pre-')>=0]

,name,refined
477,pre-sildenafil,
600,pre-abciximab,
789,pre-carvedilol,
857,pre-sirolimus-eluting,pre-sirolimus
894,pre-docetaxel,
986,pre-ivermectin,
1147,pre-mibefradil,
1372,pre-terlipressin,
1438,pre-albumin,
1630,pre-propafenone,


In [20]:
drugs.loc[drugs.name.str.find('-prev')>=0]

,name,refined
1165,enalapril-prev,enalapril


In [21]:
drugs.loc[drugs.name.str.find('refractory')>=0]

,name,refined
346,rituximab-refractory,
1349,cisplatin-refractory,


In [22]:
drugs.loc[drugs.name.str.find('resistant')>=0]

,name,refined
280,rifampicin-resistant,rifampicin
587,tetracycline-resistant,tetracycline
734,oseltamivir-resistant,oseltamivir
842,erythromycin-resistant,erythromycin
1109,ganciclovir-resistant,ganciclovir
1169,ceftriaxone-resistant,ceftriaxone
1223,clindamycin-resistant,clindamycin
1855,cefotaxime-resistant,cefotaxime
2045,meticillin-resistant,meticillin
2175,vancomycin-resistant,vancomycin


In [23]:
drugs.loc[drugs.name.str.find('insensitive')>=0]

,name,refined
1403,adenosine-insensitive,adenosine


## 정체된 목록입니다.

In [18]:
drugs.loc[drugs.refined.str.strip()!=''].to_csv('./refined.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분
- 약물명 중에 '+', ':' 등 표기 되어있는 경우가 있는데, 두 가지 약물이 합쳐져서 쓰여있는 경우도 있는 것 같은데, 이 경우 어떻게 약물명을 정제하면 좋을까요?

In [19]:
# : 등으로 약물이 나눠진 경우
drugs[drugs['refined'] == ''].loc[drugs.name.str.find(':')>=0]

,name,refined
50,albumin:creatinine,
458,a2:retinol,


In [20]:
# + 등으로 약물이 나눠진 경우
drugs[drugs['refined'] == ''].loc[drugs.name.str.find('+')>=0]

,name,refined
63,buserelin+cpa,
136,daily+oxaliplatin,
323,tamoxifen+late,
399,polychemotherapy+tamoxifen,
426,rads+prednisone,
463,tiotropium+roflumilast,
470,gy+capecitabine,
727,azt+3tc+abacavir+nfv,
752,acetate+prednisone,
759,alteplase+sk,


- 추가적으로 정제하는데 어려움을 겪고 있는 약물용어가 450개 정도 됩니다. 한번 봐주시면서 어떤 접근 방식이 좋을 지 알려주시면 감사하겠습니다.

In [21]:
drugs[drugs['refined'] == ''].shape

(451, 2)

In [22]:
drugs[drugs['refined'] == ''].to_csv('./need_to_refined.csv', sep=',')

In [23]:
drugs.describe()

,name,refined
count,2276,2276
unique,2085,1572
top,heparin,
freq,4,451
